In [1]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!ngrok authtoken 2ViAHCErBhhoXFs2st8ykv0qC5y_4gr6S4N1RKFuHx7AQN8k8

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15964 sha256=2a5f1f87a93f4a452cee7c93a75d63ced8ea60f59350c0c78c2b5787cd7308a3
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import torch
from transformers import pipeline
from accelerate import init_empty_weights
import bitsandbytes as bnb

In [4]:
oracle = pipeline(model="facebook/nllb-200-distilled-600M", tokenizer="facebook/nllb-200-distilled-600M",device="cuda",torch_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [5]:
def replace_8bit(model,threshold=6.0,model_not_to_convert="lm_head"):
  for name,module in model.named_children():
    if len(list(module.children())) > 0:
      replace_8bit(module,threshold,model_not_to_convert)

    if isinstance(module,torch.nn.Linear) and name != model_not_to_convert:
      with init_empty_weights():
            model._modules[name] = bnb.nn.Linear8bitLt(
            module.in_features,
            module.out_features,
            module.bias is not None,
            has_fp16_weights=False,
            threshold=threshold,
        )
    return model

In [6]:
oracle.model = replace_8bit(oracle.model)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
oracle.tokenizer.padding_side = "left"

In [ ]:
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/translate",methods=["GET"])
def hello():
    args = request.args
    if not args or args.get("English_text") == "":
      return jsonify({"Code":200,"response":"Please input some text"})
    inputs = oracle.tokenizer(args.get("English_text"),return_tensors="pt")
    inputs.to("cuda")
    translated_tokens = oracle.model.generate(**inputs, forced_bos_token_id=oracle.tokenizer.lang_code_to_id["hin_Deva"], max_length=120)
    text = oracle.tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    torch.cuda.empty_cache()
    return jsonify({"Hindi_text":text})
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://ed7d-34-28-13-43.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Sep/2023 01:28:35] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Sep/2023 01:28:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Sep/2023 01:29:14] "GET /translate?English_text=Hello+how+are+you? HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Sep/2023 01:30:55] "GET /translate?English_text=Hello+how+are+you?+I+am+doing+great+today HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Sep/2023 01:31:24] "GET /translate?English_text=ByteLinguists+participated+in+SIH HTTP/1.1" 200 -
